In [1]:
import re
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
webpage = "https://www.anvur.it/attivita/classificazione-delle-riviste/classificazione-delle-riviste-ai-fini-dellabilitazione-scientifica-nazionale/elenchi-di-riviste-scientifiche-e-di-classe-a/"

In [3]:
headers = {
    'User-Agent': 'Anvur Journal Acquisition',
    "Mail": "luigi.palumbo@unitus.it"
}

In [4]:
with requests.get(webpage, headers=headers) as res:
    response = BeautifulSoup(res.text, "html.parser")

In [5]:
link_list = [
    {
        "area": item.get_text(strip=True),
        "filename": item.get("href").split("/")[-1],
        "link": item.get("href")
    }
    for item in
    response.find_all("a", {"href": re.compile("pdf$")})
]

In [6]:
link_classe_a = [
    item for item
    in link_list
    if item.get("area").endswith("classe A")
]

In [7]:
link_scientifici = [
    item for item
    in link_list
    if item not in link_classe_a
]

In [38]:
link_df = pd.DataFrame(link_list)

In [39]:
link_df.to_csv("data/anvur_journals.csv", index=False)

In [48]:
for item in link_list:
    time.sleep(2)
    with requests.get(item.get("link"), headers=headers) as res:
        with open("data/{}".format(item.get("filename")), "wb") as f:
            f.write(res.content)

In [16]:
scimago_link = "https://www.scimagojr.com/journalrank.php?out=xls"

## Download ranking from Scimago

In [17]:
with requests.get(scimago_link, headers=headers) as res:
    with open("data/scimago.csv", "wb") as f:
        f.write(res.content)

In [3]:
scimago_df = pd.read_csv("data/scimago.csv", sep=";")

Categories and Areas to dummies

In [10]:
scimago_df["Categories"].head(10).str.get_dummies(sep="; ")

,Applied Microbiology and Biotechnology (Q1),"Biochemistry, Genetics and Molecular Biology (miscellaneous) (Q1)",Bioengineering (Q1),Biomaterials (Q1),Biomedical Engineering (Q1),Biotechnology (Q1),Cell Biology (Q1),Economics and Econometrics (Q1),"Electronic, Optical and Magnetic Materials (Q1)",Energy (miscellaneous) (Q1),...,Health (social science) (Q1),Health Information Management (Q1),"Health, Toxicology and Mutagenesis (Q1)",Hematology (Q1),Materials Chemistry (Q1),Medicine (miscellaneous) (Q1),Molecular Biology (Q1),Molecular Medicine (Q1),Oncology (Q1),"Surfaces, Coatings and Films (Q1)"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,1,0,0,0,0
7,1,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,1
9,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
scimago_df["Areas"].head(10).str.get_dummies(sep=", ")

,Biochemistry,Econometrics and Finance,Economics,Energy; Materials Science,Environmental Science; Health Professions; Medicine; Social Sciences,Genetics and Molecular Biology,Genetics and Molecular Biology; Chemical Engineering; Engineering; Immunology and Microbiology,Genetics and Molecular Biology; Medicine,Medicine
0,0,0,0,0,0,0,0,0,1
1,0,1,1,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0
3,1,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,1
5,1,0,0,0,0,0,0,1,0
6,0,0,0,0,1,0,0,0,0
7,1,0,0,0,0,0,1,0,0
8,0,0,0,1,0,0,0,0,0
9,0,1,1,0,0,0,0,0,0


In [6]:
scimago_df[["Title","Country"]].head(10)

,Title,Country
0,Ca-A Cancer Journal for Clinicians,United States
1,Quarterly Journal of Economics,United Kingdom
2,Nature Reviews Molecular Cell Biology,United Kingdom
3,Cell,United States
4,New England Journal of Medicine,United States
5,Nature Medicine,United Kingdom
6,MMWR Recommendations and Reports,United States
7,Nature Biotechnology,United Kingdom
8,Nature Reviews Materials,United Kingdom
9,American Economic Review,United States


## Classe A

In [8]:
import camelot

In [13]:
classe_a_test = camelot.read_pdf("data/{}".format(link_classe_a[0].get("filename")), pages="1-end")

In [15]:
classe_a_test[0].df

,0,1,2,3,4,5,6
0,TITOLO,ISSN,08/C1,08/D1,08/E1,08/E2,08/F1
1,2G,1136‐9647,A,A,A,A,A
2,ABITARE,0001‐3218,A,A,A,A,A
3,ABITARE LA TERRA,1592‐8608,A,A,A,A,A
4,ACM JOURNAL ON COMPUTING AND CULTURAL HERITAGE,1556‐4673,,,A(2017),,
5,ACOUSTICS,2624‐599X,A(2018),,,,
6,ADVANCED MATERIALS RESEARCH,1662‐8985,‡(2020),‡(2020),‡(2020),‡(2020),‡(2020)
7,ADVANCED MATERIALS RESEARCH,1022‐6680,‡(2020),‡(2020),‡(2020),‡(2020),‡(2020)
8,AGATHON,2464‐9309,A(2017),A(2017),A(2017),A(2017),
9,AGATHÓN,2532‐683X,A(2017),A(2017),A(2017),A(2017),
